# Task1

## Libraries

In [2]:
import pandas as pd
import json
import ast
import numpy as np

In [3]:
credits = pd.read_csv('../data/credits.csv')
keywords = pd.read_csv('../data/keywords.csv')
links_small = pd.read_csv('../data/links_small.csv')
links = pd.read_csv('../data/links.csv')
movies_metadata = pd.read_csv('../data/movies_metadata.csv')
ratings_small = pd.read_csv('../data/ratings_small.csv')
ratings = pd.read_csv('../data/ratings.csv')

/var/folders/vs/t6hclhb524l53jbbc2n222640000gn/T/ipykernel_58886/2131900611.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('../data/movies_metadata.csv')


In [ ]:
movies_metadata

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [ ]:
# Merge all of them to decrease confusion
credits  = credits.merge(keywords, on = 'id')
links  = links.merge(ratings_small, on = 'movieId')
links.rename(columns = {'tmdbId':'id'}, inplace = True) #Links and credits have same column id and tmdbid
links  = credits.merge(links, on = 'id')

In [6]:
movies_metadata = movies_metadata.drop([29503,35587,19730]) # as the data in those points arent clear and those indices are removed

movies_metadata['id'] = movies_metadata['id'].astype('int64') #id is an object we need to convert into int

movies_metadata = movies_metadata.merge(links,on='id')

In [7]:
# Dropping Duplicates
print(movies_metadata.duplicated().sum())
df = movies_metadata.drop_duplicates()
# Dropping unnecessary columns and rearranging them for readability
df = df[['id','title','genres','overview','rating','spoken_languages','cast','crew','vote_average','vote_count','popularity','budget','keywords','production_companies','revenue','runtime','poster_path']]

244


## Data Cleaning

In [8]:
df.shape

(99878, 17)

In [9]:
df.duplicated().sum()

np.int64(71255)

In [10]:
df = df.drop_duplicates()

df.shape

(28623, 17)

In [11]:
df.isnull().sum()

id                       0
title                    0
genres                   0
overview                13
rating                   0
spoken_languages         0
cast                     0
crew                     0
vote_average             0
vote_count               0
popularity               0
budget                   0
keywords                 0
production_companies     0
revenue                  0
runtime                  0
poster_path              6
dtype: int64

In [12]:
df.dropna(inplace=True)

We have id name for each genre in our dataset as we are most interested in the name of genre not on its id, we only keep its name by removing the extra unnecessary information

Genre / Keywords / Cast / Crew / SpokenLanguage

In [13]:
import ast

def genre_convert(data):
    genre = []
    for i in ast.literal_eval(data):
        genre.append(i['name'])
    return genre 
df['genres'] = df['genres'].apply(genre_convert)

def name_convert(data):
    keywords = []
    for i in ast.literal_eval(data):
        keywords.append(i['name'])
    return keywords 
df['keywords'] = df['keywords'].apply(name_convert)

def cast_names(data):
    actors = []
    counter = 0
    for i in ast.literal_eval(data):
        if counter != 3:
            actors.append(i['name'])
            counter += 1
    return actors 
df['cast'] = df['cast'].apply(cast_names)


def crew_names(data):
    director = []
    for i in ast.literal_eval(data):
        if i['job'] == "Director":
            director.append(i['name'])
            break
    return director
df['crew'] = df['crew'].apply(crew_names)

def spoken_lang(data):
    keywords = []
    for i in ast.literal_eval(data):
        keywords.append(i['name'])
    return keywords
df['spoken_languages'] = df['spoken_languages'].apply(spoken_lang)

def production_companies(data):
    production_companies = []
    for i in ast.literal_eval(data):
        production_companies.append(i['name'])
    return production_companies 
df['production_companies'] = df['production_companies'].apply(production_companies)

df['genres'] = df['genres'].apply(lambda x:[i.replace(" ","") for i in x])
df['cast'] = df['cast'].apply(lambda x:[i.replace(" ","") for i in x])
df['crew'] = df['crew'].apply(lambda x:[i.replace(" ","") for i in x])
df['keywords'] = df['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
df['genres'] = df['genres'].apply(lambda x:" ".join(x))
df['cast'] = df['cast'].apply(lambda x:" ".join(x))
df['crew'] = df['crew'].apply(lambda x:" ".join(x))
df['keywords'] = df['keywords'].apply(lambda x:" ".join(x))
df['spoken_languages'] = df['spoken_languages'].apply(lambda x:" ".join(x))
df['production_companies'] = df['production_companies'].apply(lambda x:" ".join(x))

In [14]:
df.head

<bound method NDFrame.head of             id                                              title  \
0          862                                          Toy Story   
1          862                                          Toy Story   
2          862                                          Toy Story   
3          862                                          Toy Story   
5          862                                          Toy Story   
...        ...                                                ...   
100104  402672                                       Mohenjo Daro   
100105  391698  The Beatles: Eight Days a Week - The Touring Y...   
100106   10991                      Pokémon: Spell of the Unknown   
100110   12600        Pokémon 4Ever: Celebi - Voice of the Forest   
100114  265189                                      Force Majeure   

                                                   genres  \
0                                 Animation Comedy Family   
1                  

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28604 entries, 0 to 100114
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    28604 non-null  int64  
 1   title                 28604 non-null  object 
 2   genres                28604 non-null  object 
 3   overview              28604 non-null  object 
 4   rating                28604 non-null  float64
 5   spoken_languages      28604 non-null  object 
 6   cast                  28604 non-null  object 
 7   crew                  28604 non-null  object 
 8   vote_average          28604 non-null  float64
 9   vote_count            28604 non-null  float64
 10  popularity            28604 non-null  object 
 11  budget                28604 non-null  object 
 12  keywords              28604 non-null  object 
 13  production_companies  28604 non-null  object 
 14  revenue               28604 non-null  float64
 15  runtime               2

In [16]:
df.to_csv('../data/movies.csv',index=False)

In [17]:
df_act = pd.read_csv("../data/movies.csv")

df_act.head(5)

,id,title,genres,overview,rating,spoken_languages,cast,crew,vote_average,vote_count,popularity,budget,keywords,production_companies,revenue,runtime,poster_path
0,862,Toy Story,Animation Comedy Family,"Led by Woody, Andy's toys live happily in his ...",3.0,English,TomHanks TimAllen DonRickles,JohnLasseter,7.7,5415.0,21.946943,30000000,jealousy toy boy friendship friends rivalry bo...,Pixar Animation Studios,373554033.0,81.0,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1,862,Toy Story,Animation Comedy Family,"Led by Woody, Andy's toys live happily in his ...",4.0,English,TomHanks TimAllen DonRickles,JohnLasseter,7.7,5415.0,21.946943,30000000,jealousy toy boy friendship friends rivalry bo...,Pixar Animation Studios,373554033.0,81.0,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg
2,862,Toy Story,Animation Comedy Family,"Led by Woody, Andy's toys live happily in his ...",5.0,English,TomHanks TimAllen DonRickles,JohnLasseter,7.7,5415.0,21.946943,30000000,jealousy toy boy friendship friends rivalry bo...,Pixar Animation Studios,373554033.0,81.0,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg
3,862,Toy Story,Animation Comedy Family,"Led by Woody, Andy's toys live happily in his ...",2.0,English,TomHanks TimAllen DonRickles,JohnLasseter,7.7,5415.0,21.946943,30000000,jealousy toy boy friendship friends rivalry bo...,Pixar Animation Studios,373554033.0,81.0,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg
4,862,Toy Story,Animation Comedy Family,"Led by Woody, Andy's toys live happily in his ...",3.5,English,TomHanks TimAllen DonRickles,JohnLasseter,7.7,5415.0,21.946943,30000000,jealousy toy boy friendship friends rivalry bo...,Pixar Animation Studios,373554033.0,81.0,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg
